# Deep learning
---------------------------

This example shows how to use ATOM to train and validate a Convolutional Neural Network implemented with [Keras](https://keras.io/) using [scikeras](https://www.adriangb.com/scikeras/refs/heads/master/index.html).

Import the MNIST dataset from [keras.datasets](https://keras.io/api/datasets/mnist/). This is a well known image dataset whose goal is to classify handwritten digits.

## Using scikeras

In [15]:
# Disable annoying tf warnings
import logging
import tensorflow as tf
tf.get_logger().setLevel(logging.ERROR)

from atom import ATOMClassifier, ATOMModel
from sklearn.preprocessing import FunctionTransformer
from skopt.space.space import Categorical

from scikeras.wrappers import KerasClassifier
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, Dropout

In [16]:
# Create the convolutional neural network
class ConvNN(KerasClassifier):
    """Convolutional neural network model."""

    @property
    def feature_encoder(self):
        """Convert the 2d input to the image's format (len(X), 28, 28, 1)."""
        return FunctionTransformer(
            func=lambda X: X.reshape(X.shape[0], 28, 28, 1),
        )

    @staticmethod
    def _keras_build_fn():
        """Create the model's architecture."""
        model = Sequential()
        model.add(
            Conv2D(
                filters=64,
                kernel_size=3,
                activation="relu",
                input_shape=(28, 28, 1),
            )
        )
        model.add(Conv2D(filters=64, kernel_size=3, activation="relu"))
        model.add(Flatten())
        model.add(Dense(units=10, activation="softmax"))
        model.compile(
            optimizer="adam",
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"],
        )

        return model

In [17]:
# Convert the model to an ATOM model
model = ATOMModel(
    estimator=ConvNN(epochs=5, verbose=0),
    acronym="CNN",
    fullname="Neural network",
)

In [18]:
# Download the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Flatten data to follow sklearn's API (2d input)
X_train = X_train.reshape(len(X_train), -1)
X_test = X_test.reshape(len(X_test), -1)

data = (X_train, y_train), (X_test, y_test)

In [19]:
atom = ATOMClassifier(*data, n_rows=0.1, n_jobs=6, verbose=2, random_state=1)

<< ================== ATOM ================== >>
Algorithm task: multiclass classification.
Parallel processing with 6 cores.

Dataset stats ==================== >>
Shape: (7000, 785)
Memory: 5.50 MB
Scaled: False
Outlier values: 42381 (0.9%)
-------------------------------------
Train set size: 6000
Test set size: 1000
-------------------------------------
|   |     dataset |       train |        test |
| - | ----------- | ----------- | ----------- |
| 0 |   666 (1.1) |   557 (1.0) |   109 (1.4) |
| 1 |   803 (1.3) |   696 (1.3) |   107 (1.3) |
| 2 |   704 (1.1) |   602 (1.1) |   102 (1.3) |
| 3 |   730 (1.2) |   636 (1.2) |    94 (1.2) |
| 4 |   668 (1.1) |   575 (1.1) |    93 (1.2) |
| 5 |   626 (1.0) |   546 (1.0) |    80 (1.0) |
| 6 |   727 (1.2) |   627 (1.1) |   100 (1.2) |
| 7 |   735 (1.2) |   624 (1.1) |   111 (1.4) |
| 8 |   678 (1.1) |   576 (1.1) |   102 (1.3) |
| 9 |   663 (1.1) |   561 (1.0) |   102 (1.3) |



In [ ]:
# Like any other model, we can define custom dimensions for the bayesian optimization
atom.run(
    models=ConvNN(),
    metric="f1_weighted",
    n_calls=5,
    bo_params={
        "dimensions": [Categorical([64, 128, 256, 512], name="batch_size")],
        "max_time": 120,
        "cv":3,
    }
)


Training ========================= >>
Models: CNN
Metric: f1_weighted


Running BO for ConvNN...
| call             | batch_size | f1_weighted | best_f1_weighted |    time | total_time |
| ---------------- | ---------- | ----------- | ---------------- | ------- | ---------- |
| Initial point 1  |        512 |      0.4709 |           0.4709 | 15.742s |    15.747s |
| Initial point 2  |        512 |      0.4709 |           0.4709 |  0.002s |    15.749s |

Exception encountered while running the CNN model. Removing model from pipeline. 
BrokenProcessPool: Could not un-serialize the CNN model to send it to the workers. This usually happens when the model is unpickable. Try using one of the predefined models, n_jobs=1 or bo_params={'cv': 1}.


## Analyze the results

In [20]:
# Use the prediction methods like any other model
atom.cnn.predict_proba(X_train)

AttributeError: ATOMClassifier object has no attribute cnn.

In [ ]:
# Or make plots...
atom.cnn.plot_confusion_matrix()